# 데이터 전처리

In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm_notebook

In [2]:
route_station = pd.read_csv('./data/PJT001_routestationinfo.csv')

## 1. only_hw_busroute.csv
'화성시'만 지나는 버스 데이터 추출  

- 방법 : 정류장 고유번호 9자리 중 앞 3자리가 '233'와 같은 행만 추출하고자 함.

In [3]:
only_hw_busroute = pd.DataFrame()

for i in tqdm_notebook(range(len(route_station))):
    if route_station.at[i, 'station_id'].astype(str)[:3] == '233':
        only_hw_busroute = only_hw_busroute.append(route_station.iloc[[i]])

'화성시'만 지나는 버스 데이터 csv file로 제작

In [4]:
# '화성시'만 지나는 버스 데이터 csv file로 제작
only_hw_busroute.to_csv('./data/only_hw_busroute.csv')

## 2. all_hw_busroute.csv
'화성시'를 지나는 버스 노선 모두 추출  

- 방법 : 버스 노선 고유번호를 이용하여 추출하고자 함.

In [5]:
hw_bus_eigen = list(set(only_hw_busroute['pr_station_id'].unique()))

In [6]:
all_hw_busroute = pd.DataFrame()

for i in tqdm_notebook(range(len(route_station))):
    if route_station.at[i, 'pr_station_id'] in hw_bus_eigen:
        all_hw_busroute = all_hw_busroute.append(route_station.iloc[[i]])

'화성시'를 지나는 버스 노선 데이터 csv file로 제작

In [7]:
# '화성시'를 지나는 버스 노선 데이터 csv file로 제작
all_hw_busroute.to_csv('./data/all_hw_busroute.csv')

## 3. merge_hw_RouteStation.csv
'화성시'를 지나는 버스 노선에 대해 정류소 정보를 merge한 후, 노선분류  
노선 : 마을, 시내, 광역, 시외, 공항으로 구분

In [8]:
# 위 과정에서 작성한 '화성시'를 지나는 버스 노선 데이터 로드
all_hw_busroute = pd.read_csv('./data/all_hw_busroute.csv')
# 필요 없는 column 제거
busroute_essential = all_hw_busroute.drop(['Unnamed: 0', 'seq', 'mobile_no'], axis = 1)
# 변수명 직관적으로 변경
busroute_essential.rename(columns = {'station_nm' : '정류소명', 'station_id' : '표준정류장ID'},inplace = True)
# 버스 정류장 테이블 로드
stationinfo = pd.read_csv('./data/PJT001_stations_table.csv')
# 필요없는 열 제거
stationinfo_essential = stationinfo.drop(['중앙차로여부', '모바일정류장ID', '이비카드정류장ID', '정류소영문명'], axis = 1)
# 표준정류장ID 기준으로 merge
total = pd.merge(busroute_essential, stationinfo_essential, on = '표준정류장ID')

직접 노선을 찾아 노선을 분류

In [9]:
bus_classification = pd.read_csv('./data/노선분류.txt', sep = '\t', encoding = 'CP949')

In [10]:
Village = bus_classification['마을']
Town = bus_classification['시내']
Urban = bus_classification['광역']
Countryside = bus_classification['시외']
Airport = bus_classification['공항']

Countryside.dropna(inplace = True)


Countryside = Countryside.astype(int).astype(str)

Village.dropna(inplace = True) # 138
Town.dropna(inplace = True) # 136
Urban.dropna(inplace = True) # 28
Countryside.dropna(inplace = True) # 2
Airport.dropna(inplace = True) # 1

In [11]:
for i in range(len(total)):
    if total.at[i, 'bus_line_no'] in list(Village) :
        total.at[i,'버스노선구분'] = '마을'
    
    elif total.at[i, 'bus_line_no'] in list(Town) :
        total.at[i,'버스노선구분'] = '시내'
    
    elif total.at[i,'bus_line_no'] in list(Urban) :
        total.at[i,'버스노선구분'] = '광역'
    
    elif total.at[i, 'bus_line_no'] in list(Countryside) :
        total.at[i,'버스노선구분'] = '시외'
    
    elif total.at[i, 'bus_line_no'] in list(Airport) :
        total.at[i,'버스노선구분'] = '공항'
    
    else :
        total.at[i,'버스노선구분'] = '?'

bus_line_no가 중복되는 경우 종종 발생

In [12]:
temp = total

In [13]:
# 화성시 버스 번호들만 추출하여 temp_bus 변수에 저장
temp_bus = temp['bus_line_no'].unique()

In [14]:
# 화성시 버스 번호가 중복되는 해당 버스의 고유ID만 hw_overlap_bus_eigen 변수에 저장
hw_overlap_bus_eigen = []

for i in temp_bus:
    if len(temp['pr_station_id'][temp['bus_line_no'] == i].unique()) > 1:
        hw_overlap_bus_eigen.append(list(temp['pr_station_id'][temp['bus_line_no'] == i].unique()))

In [15]:
hw_overlap_bus_eigen

[[228000204, 233000081],
 [241317009, 233000274, 241485033],
 [241489008, 241205008],
 [241483002, 233000019],
 [241335006, 233000058],
 [233000086, 241491008],
 [241205001, 233000045],
 [233000025, 241205007],
 [233000026, 241317002],
 [233000044, 241324001],
 [233000060, 241323002],
 [233000080, 241483018],
 [233000096, 241324002],
 [233000056, 241205003],
 [241485006, 241323004],
 [241485018, 233000064]]

In [16]:
# hw_overlap_bus_eigen를 리스트로 변환하여 hw_overlap_bus_pr_id 변수에 저장
hw_overlap_bus_pr_id = sum(hw_overlap_bus_eigen, [])

In [17]:
# 화성시 버스 번호가 중복되는 해당 버스의 고유ID의 노선만 추출하여 데이터프레임화
hw_overlap_bus = pd.DataFrame()

for i in tqdm_notebook(range(len(temp))):
    if temp['pr_station_id'][i] in hw_overlap_bus_pr_id:
        hw_overlap_bus = hw_overlap_bus.append(temp.iloc[[i]])

In [18]:
# 화성시 버스 번호가 중복되는 버스 번호 살펴보기
hw_overlap_bus['bus_line_no'].unique()

array(['24', '39', '9-1', '5-2', '7', '38', '6-3', '9', '19', '17',
       '11-2', '21', '27', '7-1', '12', '10'], dtype=object)

In [19]:
# 화성시 버스 번호가 중복되는 데이터 정리
for i in range(len(hw_overlap_bus_eigen)):
    print('{0:<4} : {1}'.format(hw_overlap_bus['bus_line_no'].unique()[i], hw_overlap_bus_eigen[i]))

24   : [228000204, 233000081]
39   : [241317009, 233000274, 241485033]
9-1  : [241489008, 241205008]
5-2  : [241483002, 233000019]
7    : [241335006, 233000058]
38   : [233000086, 241491008]
6-3  : [241205001, 233000045]
9    : [233000025, 241205007]
19   : [233000026, 241317002]
17   : [233000044, 241324001]
11-2 : [233000060, 241323002]
21   : [233000080, 241483018]
27   : [233000096, 241324002]
7-1  : [233000056, 241205003]
12   : [241485006, 241323004]
10   : [241485018, 233000064]


In [20]:
# bus_line_no 가 중복될 경우 구분
# 10 
total.loc[(total['pr_station_id'] == 233000064), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241485018), '버스노선구분'] = '마을'

# 11-2
total.loc[(total['pr_station_id'] == 233000062), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241323002), '버스노선구분'] = '마을'

# 17
total.loc[(total['pr_station_id'] == 233000044), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241324001), '버스노선구분'] = '마을'

# 19 
total.loc[(total['pr_station_id'] == 233000026), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241317002), '버스노선구분'] = '마을'

# 27
total.loc[(total['pr_station_id'] == 233000096), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241324002), '버스노선구분'] = '마을'

# 38
total.loc[(total['pr_station_id'] == 233000086), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241491008), '버스노선구분'] = '마을'

# 39
total.loc[(total['pr_station_id'] == 233000074), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241317009), '버스노선구분'] = '마을'
total.loc[(total['pr_station_id'] == 241485033), '버스노선구분'] = '마을'

# 6-3
total.loc[(total['pr_station_id'] == 233000045), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241205001), '버스노선구분'] = '마을'

# 7
total.loc[(total['pr_station_id'] == 233000058), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241335006), '버스노선구분'] = '마을'

# 9 
total.loc[(total['pr_station_id'] == 233000025), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241205007), '버스노선구분'] = '마을'

# 7-1
total.loc[(total['pr_station_id'] == 233000056), '버스노선구분'] = '시내'
total.loc[(total['pr_station_id'] == 241205003), '버스노선구분'] = '마을'

In [21]:
# 구분하지 않은 버스 노선은 없는지 확인
total[total['버스노선구분'] == '?']['bus_line_no'].unique()

array(['29', '11-3'], dtype=object)

In [22]:
# 29, 11-3번 노선은 없는 노선이라 제외
total = total[total['버스노선구분'] != '?']
total.drop('정류소명_y', axis = 1, inplace = True)
total.rename(columns = {'정류소명_x':'정류소명'}, inplace = True)

In [23]:
station_loc = pd.read_csv('./data/정류장위치정보(결측치).txt', sep = '\t', error_bad_lines=False)
# 3 개가 안들어감
station_loc.rename(columns={'장소명' : '정류소명', '위도' : 'WGS84위도', '경도' : 'WGS84경도'}, inplace = True)
station_loc

b'Skipping line 182: expected 3 fields, saw 4\n'


,정류소명,WGS84위도,WGS84경도
0,마니커입구삼거리,37.150824,127.204074
1,자이파밀리에,37.159393,127.105963
2,롯데캐슬.제일풍경채,37.157945,127.108970
3,에이스카운티,37.202422,127.209194
4,백학IC(경유),37.225456,126.879367
...,...,...,...
188,봉담한신아파트후문,37.215682,126.949127
189,봉담초,37.224103,126.947034
190,우방아파트앞,37.201570,127.007198
191,신한미지엔정문,37.201569,127.006386


In [24]:
for i in range(len(station_loc)):
    tmp = station_loc.loc[i,"정류소명"]
    total.loc[total['정류소명']==tmp,"WGS84위도"] = station_loc.loc[i,"WGS84위도"]
    total.loc[total['정류소명']==tmp,"WGS84경도"] = station_loc.loc[i,"WGS84경도"]

In [25]:
total[total['WGS84위도'].isna()]['정류소명'].unique()

array(['양우내안애(경유)', '용건릉(경유)', '서연초.제일풍경채', '삼부실교차로(경유)', '장전출발(경유)',
       '장전도착(경유)', '신창2차비바패밀리', '원룸단지앞'], dtype=object)

In [ ]:
# (경유) 정류장은 결측치 유지
# 필요한 정류장은 직접 위 경도 확인

# 서연초.제일풍경채 37.159319,127.1109993
# 신창2차비바패밀리 37.208789,127.0453479
# 원룸단지앞 37.215715,127.0376103

In [26]:
total.loc[total['정류소명']=='서연초.제일풍경채',"WGS84위도"] = 37.159319
total.loc[total['정류소명']=='서연초.제일풍경채',"WGS84경도"] = 127.1109993

total.loc[total['정류소명']=='신창2차비바패밀리',"WGS84위도"] = 37.208789
total.loc[total['정류소명']=='신창2차비바패밀리',"WGS84경도"] = 127.0453479

total.loc[total['정류소명']=='원룸단지앞',"WGS84위도"] = 37.215715
total.loc[total['정류소명']=='원룸단지앞',"WGS84경도"] = 127.0376103

In [ ]:
# .csv 데이터생성
total.to_csv('./data/merge_hw_RouteStation.csv')

## ODdata3, ODdata3_1

2018년 7월 3일 트립체인 OD데이터 전처리

- ODdata3 : 2018년 7월 3일 수집건수 분류 X 데이터
- ODdata3_1 : 2018년 7월 3일 수집건수 = 1 데이터


## 4. ODdata3

In [28]:
df_tripchain = pd.read_csv('./data/PJT001_TripChain.csv')

### column명 재설정 필요

In [29]:
# mydict[new_key] = mydict.pop(old_key) 활용

keys = df_tripchain.keys()
keys[1][1:-1]
for i in range(1, len(keys)-1):
    df_tripchain[keys[i][3:-3]] = df_tripchain.pop('\''+keys[i][1:-1]+'\'')

In [30]:
df_tripchain['암호화카드번호'] = df_tripchain.pop('암호화카드번호||\'')
df_tripchain['트립체인완료코드'] = df_tripchain.pop('\'||트립체인완료코드')

날짜 포멧이 제 각각  
모두 datetime 포맷으로 바꾸면, 용량이 너무 커질 것이므로 필요한 열인 최초 하차일시만 datetime 형태로 바꿈.

In [31]:
df_tripchain['최종하차일시'] = pd.to_datetime(df_tripchain['최종하차일시'], errors = 'coerce')

버스 이용시간을 버스 최종하차 기준으로 잡으려고 한다.  
따라서 최종하차일시가 결측된 데이터는 데이터에서 제거해준다.

In [32]:
before = len(df_tripchain)
df_tripchain.dropna(subset = ['최종하차일시'], inplace = True)
after = len(df_tripchain)

print("{} -> {} ({:.2f})".format(before, after, (before - after)/before))

1048575 -> 1036746 (0.01)


1% 데이터만 소실됐다.

### 특정 날짜 데이터

몇일의 데이터가 있는지 확인

In [33]:
df_tripchain['최종하차일시'].dt.day.value_counts().sort_index()

1    177387
2    360373
3    399390
4     98971
5       625
Name: 최종하차일시, dtype: int64

2018년 7월 1일 ~ 5일 까지의 데이터가 있음을 확인 (하차날짜 기준)  
이 중에서, 데이터가 가장 많은 **2018년 7월 3일** 데이터만 사용  

In [34]:
df = df_tripchain[df_tripchain['최종하차일시'].dt.day == 3].reset_index(drop = True)

이 데이터의 핵심은 승차와 하차에 대한 정보가 담겨있는 것이다.  
위 데이터를 잘 살펴보면 최초 승차역 ID가 대부분 결측값이지만, 승차역 ID1가 대체로 동일함을 알 수 있다.  
따라서, 최초승차역 ID의 값들을 다음과 같이 채워준다.

In [35]:
df['최초승차역ID'] = df['승차역ID1']

승차역과 하차역에 대한 결측치를 살펴본다.

In [36]:
print(df['최초승차역ID'].isna().sum())
print(df['최종하차역ID'].isna().sum())

16
1135


전체에 비하면 매우 적은양이기 때문에 결측 값을 가지는 데이터는 모두 드랍.

In [37]:
before = len(df)
df.dropna(subset=['최초승차역ID', '최종하차역ID'], inplace=True)
after = len(df)

print("{} -> {} ({:.2f})".format(before, after, (before-after)/before))

399390 -> 398249 (0.00)


드랍된 데이터가 1% 미만임

### OD 시각화 by 정류장
### 정류장 테이블과 합치기
승차 정류장에서 하차 정류장으로 가는 이동을 시각화 

데이터 프레임 만들기

In [39]:
### 수집건수 분류 x ###
df_from_to = df.groupby(['최초승차역ID', '최종하차역ID']).size().to_frame().reset_index()
df_from_to.rename({0: "이용량"}, axis=1, inplace=True)
df_from_to.head(5)

,최초승차역ID,최종하차역ID,이용량
0,150.0,4100042.0,1
1,150.0,4100051.0,1
2,150.0,4100068.0,1
3,150.0,4100077.0,1
4,150.0,4100083.0,1


In [40]:
df_stations = pd.read_csv('./data/PJT001_stations_table.csv')

In [41]:
### 수집건수 분류 x merge ###
df_from_to = pd.merge(df_from_to,
            df_stations[['이비카드정류장ID', '정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
            left_on='최초승차역ID', right_on='이비카드정류장ID', how='left')

df_from_to = pd.merge(df_from_to, 
            df_stations[['이비카드정류장ID', '정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
            left_on='최종하차역ID', right_on='이비카드정류장ID', how='left',
            suffixes=('_승차', '_하차'))
df_from_to.head(5)

,최초승차역ID,최종하차역ID,이용량,이비카드정류장ID_승차,정류소명_승차,표준정류장ID_승차,WGS84위도_승차,WGS84경도_승차,관할관청_승차,이비카드정류장ID_하차,정류소명_하차,표준정류장ID_하차,WGS84위도_하차,WGS84경도_하차,관할관청_하차
0,150.0,4100042.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,150.0,4100051.0,1,NaN,NaN,NaN,NaN,NaN,NaN,4100051.0,동부출장소.병점초등학교,233000708.0,37.208800,127.034433,경기도 화성시
2,150.0,4100068.0,1,NaN,NaN,NaN,NaN,NaN,NaN,4100068.0,호계2동주민센터,209000126.0,37.382017,126.952783,경기도 안양시
3,150.0,4100077.0,1,NaN,NaN,NaN,NaN,NaN,NaN,4100077.0,비상활주로.곡반정동입구,201000040.0,37.233000,127.021367,경기도 수원시
4,150.0,4100083.0,1,NaN,NaN,NaN,NaN,NaN,NaN,4100083.0,수원버스터미널,201000009.0,37.250950,127.019550,경기도 수원시


이 중에는 위치정보에 결측치가 있을 수도 있다.

In [42]:
### 수집건수 분류 x 결측치 확인 ###
cols = [col for col in df_from_to.columns if "WGS" in col]
missing_num = df_from_to[cols].isna().any(axis=1).sum()

print("{} / {} ({:.2f}%)".format(missing_num, len(df_from_to), missing_num/len(df_from_to)))

72480 / 186048 (0.39%)


약 39%의 결측치가 있다.  
지도에 시각화 시킬 수 없으므로, 드랍시키도록 한다.

In [43]:
### 수집건수 분류 x 결측치 제거 ###
df_from_to = df_from_to[~df_from_to[cols].isna().any(axis=1)]
df_from_to.tail(5)
#관할관청 하차 경기도 화성시만 
#관할관청 승차 경기도 화성시만 

,최초승차역ID,최종하차역ID,이용량,이비카드정류장ID_승차,정류소명_승차,표준정류장ID_승차,WGS84위도_승차,WGS84경도_승차,관할관청_승차,이비카드정류장ID_하차,정류소명_하차,표준정류장ID_하차,WGS84위도_하차,WGS84경도_하차,관할관청_하차
185970,9196637.0,4100565.0,1,9196637.0,율동공원사거리,206000346.0,37.383900,127.150233,경기도 성남시,4100565.0,파크타운.양지마을,206000183.0,37.373967,127.119567,경기도 성남시
185971,9196637.0,4100566.0,1,9196637.0,율동공원사거리,206000346.0,37.383900,127.150233,경기도 성남시,4100566.0,중앙공원.샛별마을우방아파트,206000184.0,37.372633,127.126183,경기도 성남시
185972,9196637.0,4116808.0,1,9196637.0,율동공원사거리,206000346.0,37.383900,127.150233,경기도 성남시,4116808.0,여권민원실.풍림아파트,202000054.0,37.287450,127.033083,경기도 수원시
185973,9196637.0,4150166.0,1,9196637.0,율동공원사거리,206000346.0,37.383900,127.150233,경기도 성남시,4150166.0,벽산아파트,228001144.0,37.334750,127.108717,경기도 용인시
185974,9196865.0,4108038.0,1,9196865.0,인정피렌체아파트,228001295.0,37.246183,127.229500,경기도 용인시,4108038.0,망포블루밍.삼정라츠아파트,203000147.0,37.237183,127.059267,경기도 수원시


In [44]:
### 수집건수 분류 x 데이터생성 ###
df_from_to.to_csv('./data/ODdata3.csv')

## 5. ODdata3_1

In [45]:
### 수집건수 == 1 환승 없는 자료 ###

# 수집건수 1
df_from_to1 = df[df['수집건수'] == 1].groupby(['최초승차역ID', '최종하차역ID', '버스노선ID1']).size().to_frame().reset_index()
df_from_to1.rename({0: "이용량"}, axis=1, inplace=True)
df_from_to1

,최초승차역ID,최종하차역ID,버스노선ID1,이용량
0,4100002.0,4100004.0,41027009.0,1
1,4100002.0,4100020.0,41027009.0,1
2,4100002.0,4100022.0,41020001.0,1
3,4100002.0,4100024.0,41027009.0,1
4,4100002.0,4100030.0,41027009.0,1
...,...,...,...,...
76211,9199422.0,4170973.0,41074080.0,1
76212,9199422.0,4176946.0,41074080.0,8
76213,9199422.0,4197464.0,41074080.0,1
76214,9199422.0,4197590.0,41074080.0,1


In [49]:
# 12월 3일 제공자료  버스루트매핑 자료 로드
routemapping = pd.read_csv('./data/PJT001_routestationmapping.csv')

In [50]:
df_from_to1 = pd.merge(df_from_to1, 
             df_stations[['이비카드정류장ID', '정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
             left_on='최초승차역ID', right_on='이비카드정류장ID', how='left')

df_from_to1 = pd.merge(df_from_to1, 
             df_stations[['이비카드정류장ID', '정류소명', '표준정류장ID', 'WGS84위도', 'WGS84경도', '관할관청']],
             left_on='최종하차역ID', right_on='이비카드정류장ID', how='left',
             suffixes=('_승차', '_하차'))

df_from_to1 = pd.merge(df_from_to1,
                       routemapping[['표준노선ID','노선명']],
                       left_on = '버스노선ID1', right_on = '표준노선ID', how = 'left',
                      )

In [51]:
cols = [col for col in df_from_to1.columns if "WGS" in col]
missing_num = df_from_to1[cols].isna().any(axis=1).sum()

print("{} / {} ({:.2f}%)".format(missing_num, len(df_from_to1), missing_num/len(df_from_to1)))

13505 / 88495 (0.15%)


In [52]:
df_from_to1 = df_from_to1[~df_from_to1[cols].isna().any(axis=1)]

In [ ]:
# 경기도 화성시, 경기도 오산시, 경기도 평택시, 경기도 안산시, 경기도 수원시
target_si =['경기도 화성시','경기도 오산시','경기도 평택시','경기도 안산시','경기도 수원시']

In [ ]:
df_from_to1 = df_from_to1[df_from_to1['관할관청_승차'].isin(target_si) & df_from_to1['관할관청_하차'].isin(target_si)]

print(df_from_to1['관할관청_승차'].unique())
print(df_from_to1['관할관청_하차'].unique())
len(df_from_to1)

In [ ]:
before = len(df_from_to1)
df_from_to1.dropna(subset=['표준노선ID', '노선명'], inplace = True)
after = len(df_from_to1)
print("{} -> {} ({:.2f})".format(before, after, (before-after)/before))

In [ ]:
# 저장
df_from_to1.to_csv('./data/ODdata3_1.csv')